## Settings

In [ ]:
setwd("~/git/hist_future_hail/") # Working directory.
source("modules/gev_functions.R") # Load helper functions.
variables <- c("hailcast_diam_max", "wind_10m") # Variables to process.
results_dir <- "/g/data/up6/tr2908/hist_future_hail/results/" # Results directory.
set.seed(42) # Random seed for reproducibility.

Set the time zone to UTC, otherwise times read in are all set to local time.

In [ ]:
Sys.setenv(TZ = "GMT")

## Read in data

In [ ]:
maxes <- read_feathers(results_dir = results_dir, pattern = "*_maxima.feather")
means <- read_feathers(results_dir = results_dir, pattern = "*_means.feather")


Convert wind speeds from m/s to km/h for analysis.

In [ ]:
maxes["wind_10m"] <- maxes["wind_10m"] * 3.6
means["wind_10m"] <- means["wind_10m"] * 3.6

Set display order for epochs.

In [ ]:
maxes$epoch = factor(maxes$epoch, levels = c("ssp245", "historical"))
means$epoch = factor(means$epoch, levels = c("ssp245", "historical"))

## Timeseries of maxima

First the timeseries of daily hail diameter maxima.

In [ ]:
options(repr.plot.width = 13, repr.plot.height = 14)
plot_ts(
    dat = maxes, ylabel = "Max daily hail diameter [mm]", var = "hailcast_diam_max",
    file = "paper/supporting info/figures/timeseries_hail.pdf"
)


In [ ]:
plot_ts(
    dat = maxes, ylabel = "Max daily 10 m wind [km/h]", var = "wind_10m",
    file = "paper/supporting info/figures/timeseries_wind.pdf"
)


In [ ]:
library(scales)
show_col(rev(hue_pal()(2)))

## Histograms of maxima

In [ ]:
fontsize <- 18
options(repr.plot.width = 13, repr.plot.height = 6)

maxes %>% ggplot(aes(x = hailcast_diam_max)) +
    facet_wrap(~domain, ncol = 3, scales = "free") +
    geom_histogram(aes(fill = epoch), alpha = 0.5, position = "identity", binwidth = 5) +
    theme_bw(fontsize) +
    scale_fill_manual(
        name = "Epoch", breaks = c("historical", "ssp245"),
        labels = c("Historical", "Future"), values = rev(hue_pal()(2))
    ) +
    labs(x = "Daily max. hail diameter [mm]", y = "Count") +
    theme(strip.background = element_blank(), strip.text = element_text(size = fontsize))

maxes %>% ggplot(aes(x = wind_10m)) +
    facet_wrap(~domain, ncol = 3, scales = "free") +
    geom_histogram(aes(fill = epoch), alpha = 0.5, position = "identity", binwidth = 5) +
    theme_bw(fontsize) +
    scale_fill_manual(
        name = "Epoch", breaks = c("historical", "ssp245"),
        labels = c("Historical", "Future"), values = rev(hue_pal()(2))
    ) +
    labs(x = expression(Daily ~ max. ~ hail * "-" * proximal ~ wind ~ group("[", km ~ h^{
        -1
    }, "]")), y = "Count") +
    theme(strip.background = element_blank(), strip.text = element_text(size = fontsize))


## Hail frequency changes


In [ ]:
day_changes = hail_day_changes(dat = maxes, out_file = "paper/supporting info/tables/hail_freq_table.tex")

## GEV fitting

In [ ]:
require(lubridate)

hist_dates = seq(ymd("1989-10-01"), ymd("2009-02-28"), by = "days")
fut_dates = seq(ymd("2080-10-01"), ymd("2100-02-28"), by = "days")

hist_dates = hist_dates[month(hist_dates) %in% c(10, 11, 12, 1, 2)]
fut_dates = fut_dates[month(fut_dates) %in% c(10, 11, 12, 1, 2)]

hist_dates = hist_dates[!(month(hist_dates) == 2 & day(hist_dates) == 29)]
fut_dates = fut_dates[!(month(fut_dates) == 2 & day(fut_dates) == 29)]

all_times = rbind(
    tibble(time = hist_dates, epoch = "historical"),
    tibble(time = fut_dates, epoch = "ssp245")
)

times = tibble()

for (d in unique(maxes$domain)) {
    times = rbind(times, cbind(all_times, list(domain = d)))
}

padded_maxes = full_join(maxes, times, by = c("epoch", "domain", "time")) %>%
    arrange(time) %>%
    select(epoch, domain, time, hailcast_diam_max, wind_10m) %>%
    mutate(
        hailcast_diam_max = replace_na(hailcast_diam_max, 0),
        wind_10m = replace_na(wind_10m, 0)
    )

In [ ]:
stopifnot(!any(maxes$wind_10m == 0))

In [ ]:
source("modules/gev_functions.R")

gp_thresholds = list("hailcast_diam_max" = 20, "wind_10m" = NA)
ev_types = list("hailcast_diam_max" = "GP", "wind_10m" = "GEV")

gev_fits <- fit_gevs(
    all_dat = padded_maxes,
    gp_thresholds = gp_thresholds,
    ev_types = ev_types
)

Plot GEV curves for each domain and epoch.

In [ ]:
plot_densities(
    gev_fits = gev_fits,
    variable = "hailcast_diam_max",
    ev_types = ev_types,
    gp_thresholds = gp_thresholds,
    label = "Maximum~hail~diameter~group('[',mm,']')",
    file = "paper/supporting info/figures/gev_dists_hail.pdf"
)


In [ ]:
plot_densities(
    gev_fits = gev_fits,
    variable = "wind_10m",
    ev_types = ev_types,
    x = seq(0, 100),
    label = "Maximum~wind~speed~group('[',km~h^{-1},']')",
    file = "paper/supporting info/figures/gev_dists_wind.pdf"
)

Plot parameters and their confidence intervals of each model in historic and ssp245 epochs; we consider two models significantly different if at least one parameter has non-overlapping confidence intervals.

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 8)
plot_params(gev_fits = gev_fits, file = "paper/supporting info/figures/fit_params.pdf")


We use the KS test to see whether a) the GEV model and training data are from different distributions, and b) whether the historic vs ssp245 models are considered significantly different. For each comparison we draw 1000 points from the modelled distribution(s) and do the KS test 100 times to get a distribution of p values. If most p values are below 0.05 then the null hypothesis that the two samples come from the same distribution can be rejected.

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 8)
plot_ks_fits(gev_fits = gev_fits, file = "paper/supporting info/figures/fit_pvals.pdf")


## Goodness of fit

QQ plots show goodness of fit between WRF-simulated and GEV-modelled values.

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 7)
plot_quantiles(
    gev_fits = gev_fits, var = "hailcast_diam_max", unit = "mm",
    file = "paper/supporting info/figures/qq_hail.pdf"
)


In [ ]:
plot_quantiles(
    gev_fits = gev_fits, var = "wind_10m", unit = "km~h^{-1}",
    file = "paper/supporting info/figures/qq_wind.pdf"
)


## Return periods

Return periods in number of hail days (extRemes always says number of years but assumes each maxima is an annual maxima). Note return levels may be nonphysical for long return periods.

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 8)
plot_return_levels(
    gev_fits = gev_fits,
    file = "paper/main/figures/return_periods.pdf"
)

## Threshold exceedence probabilities

Look at probabilities of given levels occurring in a given hail day.

In [ ]:
ret_periods <- rbind(
    periods_for_thresholds(gev_fits = gev_fits, var = "hailcast_diam_max", thresh = c(50, 100)),
    periods_for_thresholds(gev_fits = gev_fits, var = "wind_10m", thresh = c(70, 90))
)

write_feather(ret_periods, "results/return_periods.feather")

In [ ]:
ret_periods$threshold = factor(ret_periods$threshold, levels=c(50, 100, 70, 90))
ret_periods$epoch = factor(ret_periods$epoch, levels=c("historical", "ssp245"), labels=c("Historical", "Future"))
tab = tabular(Factor(domain) * Factor(epoch) ~ Factor(threshold) * identity * period, data = ret_periods)
toLatex(tab, file="paper/supporting info/tables/return_periods_table.tex")

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 6)
plot_probs(gev_fits = gev_fits, file = "paper/main/figures/threshold_probs.pdf")


In [ ]:
# Write outputs for display in paper.
write_feather(gev_fits$hail_probs, "results/hail_probs.feather")
write_feather(gev_fits$wind_probs, "results/wind_probs.feather")


Show tables of probabilities for given levels.

In [ ]:
probabilities_table(gev_fits = gev_fits, out_file = "paper/supporting info/tables/probabilities_table.tex")


## Changes in ingredients

In [ ]:
ing_changes = ingredients_changes(ings = means)
max_changes = ingredients_changes(ings = maxes)

Include hail frequency and max hail size changes in the ingredients changes table.

In [ ]:
ing_changes <- rbind(
    day_changes$t_test_disp %>%
        select(!starts_with("mean")) %>%
        select(!starts_with("sd")) %>%
        mutate(variable = "Hail days"),
    max_changes %>%
        filter(variable == "Hail size") %>%
        mutate(variable = "Max hail"),
    ing_changes %>%
        filter(variable == "Hail size") %>%
        mutate(variable = "Mean hail"),
    max_changes %>%
        filter(variable == "Wind") %>%
        mutate(variable = "Max wind"),
    ing_changes %>%
        filter(variable == "Wind") %>%
        mutate(variable = "Mean wind"),
    ing_changes %>%
        filter(variable != "Hail size") %>%
        filter(variable != "Wind")
)


Split latex table into two.

In [ ]:
ing_table_1 = ing_changes %>% filter(domain %in% c("Adelaide", "Brisbane", "Kalgoorlie"))
ing_table_2 <- ing_changes %>% filter(!(domain %in% c("Adelaide", "Brisbane", "Kalgoorlie")))

tab <- tabular(Factor(variable) ~ Heading() * identity * Heading() * Factor(domain) * Heading() *
                   (rel_change + sig + change_range), data = ing_table_1)
print(toLatex(tab, file = "paper/main/tables/ingredients_1.tex"))

tab <- tabular(Factor(variable) ~ Heading() * identity * Heading() * Factor(domain) * Heading() *
                   (rel_change + sig + change_range), data = ing_table_2)
print(toLatex(tab, file = "paper/main/tables/ingredients_2.tex"))

tab <- tabular(Factor(variable) ~ Heading() * identity * Heading() * Factor(domain) * Heading() *
                   (Format(digits = 1) * historic), data = ing_changes)
print(toLatex(tab, file = "paper/supporting info/tables/historic_means.tex"))

write_feather(ing_changes, "results/ingredient_changes.feather")

Plot correlations between variables at different cities.

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10)
domain_correlation_plot(means, plot_file = "paper/main/figures/correlations.pdf")